In [1]:
import arcpy
import os

# 设置工作空间
arcpy.env.workspace = r"D:\ArcGISProjects\workspace\sp2024\MyProject.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# 输入参数
input_feature = r"C:\Users\Runker\Desktop\ele_sb\sb_merge_data_single_split_curve.shp"
output_feature = "sb_merge_data__MultipartToSi1_rsult"
dlmc_field = "DLMC"

# 获取所有唯一的DLMC值
with arcpy.da.SearchCursor(input_feature, [dlmc_field]) as cursor:
    dlmc_values = sorted(set(row[0] for row in cursor))

print(f"发现 {len(dlmc_values)} 个不同的DLMC类型")

In [ ]:
dlmc_values


In [ ]:
# 创建第一个临时结果要素类
temp_result = "in_memory/temp_result"
arcpy.CopyFeatures_management(input_feature, temp_result)

In [ ]:
# 为每个DLMC指定不同的面积阈值
threshold_values = {
    '乔木林地': 500,
    '人工牧草地': 500,
    '其他园地': 500,
    '其他林地': 500,
    '其他草地': 500,
    '天然牧草地': 500,
    '旱地': 50,
    '果园': 50,
    '水田': 50,
    '灌木林地': 500,
    '竹林地': 500,
    '茶园': 50,
    '裸土地': 50,
    '裸岩石砾地': 50,
    '设施农用地': 50
}
default_threshold = 50
# 为每个DLMC值进行处理,若面积小于阈值则消除，没指定阈值则使用默认阈值
for index, dlmc_value in enumerate(dlmc_values):
    print(f"\n正在处理DLMC {index}/{len(dlmc_values)}: {dlmc_value}")
    # 创建要素图层
    arcpy.MakeFeatureLayer_management(temp_result, "working_layer")
    
    # 选择当前DLDM值的小于阈值面积的图斑
    where_clause = f"area < {threshold_values.get(dlmc_value, default_threshold)} AND {dlmc_field} = '{dlmc_value}'"
    arcpy.SelectLayerByAttribute_management("working_layer", "NEW_SELECTION", where_clause)
    
    # 获取选择的图斑数量
    selected_count = int(arcpy.GetCount_management("working_layer")[0])
    
    if selected_count > 0:
        print(f"  发现 {selected_count} 个小于 {threshold_values.get(dlmc_value, default_threshold)} 平方米的图斑")
        
        # 执行Eliminate操作，结果存储到新的临时要素类
        temp_eliminated = f"in_memory/temp_eliminated_{index}"
        arcpy.Eliminate_management(
            "working_layer", 
            temp_eliminated,
            "LENGTH", 
            f"{dlmc_field} <> '{dlmc_value}'",  # 保证只合并到相同DLMC类型
            None
        )
        
        # 删除上一轮的临时结果，并将新结果复制为temp_result
        arcpy.Delete_management(temp_result)
        arcpy.CopyFeatures_management(temp_eliminated, temp_result)
        arcpy.Delete_management(temp_eliminated)
        
        print("  已完成消除操作")
    else:
        print("  未发现需要处理的小图斑")
    
    # 清理临时图层
    arcpy.Delete_management("working_layer")

# 将最后的临时结果复制到最终输出要素类
arcpy.CopyFeatures_management(temp_result, output_feature)

# 删除最后的临时结果
arcpy.Delete_management(temp_result)

print(f"\n所有DLMC类型的处理已完成。最终结果保存在: {os.path.join(arcpy.env.workspace, output_feature)}")

# # 重建空间索引和统计信息
# arcpy.AddSpatialIndex_management(output_feature)
# arcpy.CalculateStatistics_management(output_feature)

print("空间索引和统计信息已更新。处理全部完成。")

In [ ]:
import arcpy
import os
import logging
from datetime import datetime

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('process.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# 设置工作空间
arcpy.env.workspace = r"D:\ArcGISProjects\workspace\sp2024\MyProject.gdb"
arcpy.env.overwriteOutput = True

def process_land_parcels(input_feature, output_feature, dlmc_field="DLMC"):
    try:
        # 记录开始时间
        start_time = datetime.now()
        logging.info(f"开始处理: {start_time}")
        
        # 为每个DLMC指定不同的面积阈值
        threshold_values = {
            '乔木林地': 500,
            '人工牧草地': 500,
            '其他园地': 500,
            '其他林地': 500,
            '其他草地': 500,
            '天然牧草地': 500,
            '旱地': 50,
            '果园': 50,
            '水田': 50,
            '灌木林地': 500,
            '竹林地': 500,
            '茶园': 50,
            '裸土地': 50,
            '裸岩石砾地': 50,
            '设施农用地': 50
        }
        default_threshold = 50

        # 创建临时工作空间
        temp_workspace = r"D:\ArcGISProjects\workspace\sp2024\temp_workspace.gdb"
        if not arcpy.Exists(temp_workspace):
            arcpy.CreateFileGDB_management(os.path.dirname(arcpy.env.workspace), "temp_workspace")
        
        # 创建第一个临时结果要素类
        temp_result = os.path.join(temp_workspace, "temp_result")
        arcpy.CopyFeatures_management(input_feature, temp_result)
        
        # 获取所有唯一的DLMC值
        with arcpy.da.SearchCursor(input_feature, [dlmc_field]) as cursor:
            dlmc_values = sorted(set(row[0] for row in cursor))
        
        logging.info(f"发现 {len(dlmc_values)} 个不同的DLMC类型")
        
        # 记录处理的总数量
        total_processed = 0
        
        # 为每个DLMC值进行处理
        for index, dlmc_value in enumerate(dlmc_values, 1):
            try:
                logging.info(f"\n正在处理DLMC {index}/{len(dlmc_values)}: {dlmc_value} "
                           f"({index/len(dlmc_values)*100:.1f}%)")
                
                # 创建要素图层
                arcpy.MakeFeatureLayer_management(temp_result, "working_layer")
                
                # 选择当前DLMC值的小于阈值面积的图斑
                where_clause = (f"SHAPE.AREA < {threshold_values.get(dlmc_value, default_threshold)} "
                              f"AND {dlmc_field} = '{dlmc_value}'")
                arcpy.SelectLayerByAttribute_management("working_layer", "NEW_SELECTION", where_clause)
                
                # 获取选择的图斑数量
                selected_count = int(arcpy.GetCount_management("working_layer")[0])
                
                if selected_count > 0:
                    logging.info(f"  发现 {selected_count} 个小于 "
                               f"{threshold_values.get(dlmc_value, default_threshold)} 平方米的图斑")
                    
                    # 执行Eliminate操作
                    temp_eliminated = os.path.join(temp_workspace, f"temp_eliminated_{index}")
                    arcpy.Eliminate_management(
                        "working_layer", 
                        temp_eliminated,
                        "LENGTH", 
                        f"{dlmc_field} <> '{dlmc_value}'",  # 保证只合并到相同DLMC类型
                        None
                    )
                    
                    # 更新临时结果
                    arcpy.Delete_management(temp_result)
                    arcpy.CopyFeatures_management(temp_eliminated, temp_result)
                    arcpy.Delete_management(temp_eliminated)
                    
                    total_processed += selected_count
                    logging.info("  已完成消除操作")
                else:
                    logging.info("  未发现需要处理的小图斑")
                
                # 清理临时图层
                arcpy.Delete_management("working_layer")
                
            except Exception as e:
                logging.error(f"处理 {dlmc_value} 时出错: {str(e)}")
                continue
        
        # 将最后的临时结果复制到最终输出要素类
        arcpy.CopyFeatures_management(temp_result, output_feature)
        
        # 清理临时文件
        arcpy.Delete_management(temp_result)
        arcpy.Delete_management(temp_workspace)
        
        # 重建空间索引和统计信息
        arcpy.AddSpatialIndex_management(output_feature)
        arcpy.CalculateStatistics_management(output_feature)
        
        # 计算处理时间
        end_time = datetime.now()
        processing_time = end_time - start_time
        
        logging.info(f"\n处理完成:")
        logging.info(f"总共处理了 {total_processed} 个小图斑")
        logging.info(f"最终结果保存在: {os.path.join(arcpy.env.workspace, output_feature)}")
        logging.info(f"总处理时间: {processing_time}")
        
        return True
        
    except Exception as e:
        logging.error(f"处理过程出错: {str(e)}")
        return False

if __name__ == "__main__":
    # 输入参数
    input_feature = r"D:\ArcGISProjects\workspace\sp2024\MyProject.gdb\sb_merge_data__MultipartToSi1_rsult"
    output_feature = "sb_merge_data__MultipartToSi1_rsultss"
    
    # 执行处理
    success = process_land_parcels(input_feature, output_feature)
    
    if success:
        logging.info("程序成功完成")
    else:
        logging.error("程序执行失败")